<a href="https://www.kaggle.com/code/loubl00m/llama-finetuned?scriptVersionId=274304212" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Loading modules

In [1]:
# Install the necessary libraries
!pip install -U transformers peft bitsandbytes torch==2.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 61.0 MB/s eta 0:00:0000:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.3 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from peft import PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

from datasets import load_dataset

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-11-07 16:54:07.873439: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-07 16:54:07.873581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-07 16:54:08.005187: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:

In [3]:
# Get the Weights & Biases keys from env and login
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token")
login(token = hf_token)

# Loading model from hugging face

In [4]:
torch_dtype = torch.float16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch_dtype,
    bnb_4bit_use_double_quant = True,
)

# Base model and Lora model paths.
base_path = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_path = "LouayYahyaoui/llama-edu"

base_model = AutoModelForCausalLM.from_pretrained(base_path, quantization_config = bnb_config, \
                                                  device_map = device)
model = PeftModel.from_pretrained(base_model, model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

adapter_config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

# Interacting with the model

In [5]:
def answer(prompt):
    if isinstance(prompt, str):
        prompt = [{'role': 'system',
        # Replace the system prompt here. Below is a simple example.
        'content': 'You are an educational assistant that helps university students understand their courses.'},
         {'role': 'user',
        'content': prompt}]
    elif isinstance(prompt, list):
        prompt = prompt[:2]
    elif isinstance(prompt, dict):
        prompt = prompt["messages"][:2]
    
    end_header = "<|end_header_id|>"
        
    input_ids = tokenizer.apply_chat_template(prompt, tokenize = True, \
                                add_generation_prompt = True, return_tensors="pt")
    
    
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.pad_token_id 

    input_ids = input_ids.to(device)
    out = model.generate(input_ids, max_new_tokens = 1024, do_sample = True, temperature = 0.6, \
    attention_mask = attention_mask, pad_token_id = pad_token_id)
    
    response = tokenizer.decode(out[0, input_ids.shape[-1]:], skip_special_tokens=True).strip()
    
    return response

In [6]:
ILOS = """CLO1.Demonstrate an understanding of computer organization and the functions of the main components (CPU, memory, I/O devices) of a computer.
CLO2.Map between various machine representations and perform basic computer arithmetic operations as implemented in digital computer hardware, including those on signed and unsigned numbers and floating point types.
CLO3.Demonstrate an understanding of computer arithmetic operations, including addition, subtraction, multiplication, division, and floating-point arithmetic.
CLO4.Demonstrate an understanding of modern processors, including logic design conventions, datapath construction, pipelining, and data/control hazards.
CLO5.Demonstrate an understanding of memory hierarchies and analyze their performance. 	Exam, Quiz, Assignment.	2
CLO6.Demonstrate an understanding of instruction set architecture, including the differences between RISC and CISC architectures, addressing modes, and assembly language programming basics.
CLO7.Demonstrate an understanding of  the CPU datapath, CPU components and their functions, the instruction execution cycle, the control unit, and CPU performance."""

In [8]:
import re
ilos = re.split(r"CLO\d+.", ILOS)[1:]
ilos

['Demonstrate an understanding of computer organization and the functions of the main components (CPU, memory, I/O devices) of a computer.\n',
 'Map between various machine representations and perform basic computer arithmetic operations as implemented in digital computer hardware, including those on signed and unsigned numbers and floating point types.\n',
 'Demonstrate an understanding of computer arithmetic operations, including addition, subtraction, multiplication, division, and floating-point arithmetic.\n',
 'Demonstrate an understanding of modern processors, including logic design conventions, datapath construction, pipelining, and data/control hazards.\n',
 'Demonstrate an understanding of memory hierarchies and analyze their performance. \tExam, Quiz, Assignment.\t2\n',
 'Demonstrate an understanding of instruction set architecture, including the differences between RISC and CISC architectures, addressing modes, and assembly language programming basics.\n',
 'Demonstrate an u

In [ ]:
# Change user prompt here
for ilo in ilos:
    prompt = "I'm struggling to " + ilo.strip() + " Provide some help."
    print(f"User prompt: {prompt}\n\nAssistant: {answer(prompt)}\n")

User prompt: I'm struggling to Demonstrate an understanding of computer organization and the functions of the main components (CPU, memory, I/O devices) of a computer. Provide some help.

Assistant: Computer organization refers to the way a computer's hardware components work together to execute instructions. The CPU (Central Processing Unit) executes instructions, memory stores data and instructions, and I/O devices interact with the user. Understanding these components helps in designing and troubleshooting computer systems. For example, the CPU fetches instructions, decodes them, and performs operations, while memory provides storage for data and instructions. I/O devices handle user input and output. Each component plays a crucial role in the computer's operation. Consider studying computer architecture and design to deepen your understanding.

User prompt: I'm struggling to Map between various machine representations and perform basic computer arithmetic operations as implemented 